In [ ]:
### Tyler E. Cork
### Stanford University
### March 3rd, 2020

In [ ]:
## User Input to Caluclate Heart Rate Pattern
indir     = '/Users/tecork/Desktop/'     # Initial Directory
N_beats   = 300;                         # Number of Beats
N_skipped = 30;                          # Number of Skipped Heart Beats

RR_mean   = 1014;                        # Mean R-R interval
RR_std    = 65.3;                        # Standard Deviation of R-R Interval
# R-R mean of 1014 and R-R Standard Deviation of 65.3 come from this paper:
# https://onlinelibrary.wiley.com/doi/abs/10.1111/anae.12317 

In [ ]:
# Initialize required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import date

# Set up Directories for storing information
today = date.today()
d = today.strftime("%Y-%b-%d")

folder_Path  = '{}_Heart_Rate_Variability_{}_Beats_{}_skipped_beats'.format(d, int(N_beats), int(N_skipped))
main_folder = os.path.join(indir, folder_Path)

Excell_Path  = 'Key_{}_Beats_{}_skipped_beats.xlsx'.format(int(N_beats), int(N_skipped))
Fig_1_Path   = 'HRV_Plot_{}_Beats_{}_skipped_beats.pdf'.format(int(N_beats), int(N_skipped))
Fig_2_Path   = 'Scanner_Trigger_{}_Beats_{}_skipped_beats.pdf'.format(int(N_beats), int(N_skipped))

Arduino_Folder = 'HRV_Arduino_Script_{}_Beats_{}_skipped_beats'.format(int(N_beats), int(N_skipped))
Arduino_Path = 'HRV_Arduino_Script_{}_Beats_{}_skipped_beats.ino'.format(int(N_beats), int(N_skipped))

if not os.path.isdir(main_folder):
    os.mkdir(main_folder)
if not os.path.isdir(os.path.join(indir, folder_Path, Arduino_Folder)):
    os.mkdir(os.path.join(indir, folder_Path, Arduino_Folder))

In [ ]:
# Generate Guassian Distriburtion of Heart Beats 
x = np.round(RR_mean + RR_std * np.random.normal(size=N_beats))
#print(np.sum(x))     # Original Duration
for i in range(0, N_skipped):
    ind = np.random.randint(1, x.shape[0]) # Index [ind] to skip (can't skip first beat)
    x[ind-1] = x[ind-1] + x[ind]           # Add skipped beat duration to previous
    x = np.delete(x, ind)                  # Remove skipped beat
#print(np.sum(x))    # Duration after Skipped beat should remain the same

In [ ]:
plt.plot(x, 'k', linewidth=2.0, label='R-R interval time')
plt.plot(np.ones(x.shape[0]) * RR_mean, '--r', linewidth=2.0,  label=r'$\mu$ = 1014 ms')
plt.plot(np.ones(x.shape[0]) * RR_mean + RR_std, '--b', linewidth=2.0,)
plt.plot(np.ones(x.shape[0]) * RR_mean - RR_std, '--b', linewidth=2.0, label=r'$\sigma$ = 65.3 ms')
plt.xlabel('Beat Number, (A.U.)')
plt.ylabel('R-R peak interval, (ms)')
plt.title('Guassian Distriburtion of Heart Beat Times')
leg = plt.legend()
plt.savefig(os.path.join(indir, folder_Path, Fig_1_Path))
plt.show()

In [ ]:
# Write Arduino File

# Initialize Arduino File
f= open(os.path.join(indir, folder_Path, Arduino_Folder, Arduino_Path),"w+")

# Create text line list for Arduino
txt_line = []

# Initial Lines of Code to Start Arduino File
txt_line.append('')
txt_line.append('const int led  =  13;        //use digital I/O pin 13\n')
txt_line.append('void setup()\n')
txt_line.append('{\n')
txt_line.append('pinMode(led,OUTPUT);         //set pin 13 to be an output\n')
txt_line.append('delay(100);                  //delay 100 milliseconds\n')
txt_line.append('}\n')
txt_line.append('void loop()\n')
txt_line.append('{\n')

# Initialize Arduinio Trigger Simulation Array
y = np.zeros(int(np.sum(x)))
start = int(0)                      # Initialize simulation start time
for ii in range(0, x.shape[0]):
    end = int(start + x[ii] - 101)  # Set end time 101 ms before trigger 
    y[start:end] = 0                # Assign 0 V to time befroe trigger
    y[end+1:end+101] = 5            # Assign 5 V trigger for 100 ms after end time 
    start = int(end + 101)          # Update start time
    
    # Append trigger on/off commands
    string_1 = "delay({});                //delay {} milliseconds\n".format(str(x[ii] - 100), (str(x[ii] - 100)))
    txt_line.append(string_1)
    string_2 = "digitalWrite(led,HIGH);      //set pin 13 HIGH\n"
    txt_line.append(string_2)
    string_3 = "delay(100);                  //delay 100 milliseconds\n"
    txt_line.append(string_3)
    string_4 = "digitalWrite(led,LOW);       //set pin 13 LOW\n"  
    txt_line.append(string_4)
    
# Close Arduino loop
txt_line.append('}\n')
txt_line.append('')

# Writes Arduino Script
f.writelines(txt_line)
f.close()

In [ ]:
plt.plot(y, 'b', linewidth=1.0, label='R-R interval time')
plt.ylim([0, 6])
plt.xlim([0,y.shape[0]])
plt.xlabel('Time, (ms)')
plt.ylabel('Volts, (V)')
plt.title('Trigger to Scanner')
plt.savefig(os.path.join(indir, folder_Path, Fig_2_Path))
plt.show()

In [ ]:
# Create Heart Rate Key
key = []
for ii in range(0, x.shape[0]):
    if x[ii] > 1.5 * RR_mean:
        note = 'Includes Skipped beat'
    else:
        note = ''
    key.append([(ii+1), x[ii], note])
    del note     
key
df = pd.DataFrame(data = key, columns=['Heart Beat Number','R-R interval time [ms]', 'Notes'])

# Writes Key to Excell File
df.to_excel(os.path.join(indir, folder_Path, Excell_Path))